# The goals/steps of this project are the following
* Perform a Histogram of Oriented Gradients(HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally you can also apply a color tranform and append binned color features, as well as histograms of color, to your HOG feature vector.
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream(start with test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

In [5]:
import numpy as np
import cv2
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob
from skimage.feature import hog
import time
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from helper import *
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
import time
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
%matplotlib qt

In [6]:
# Loading the car and notcar data
vehicles = glob.glob('vehicles/**/*.png')
non_vehicles = glob.glob('non-vehicles/**/*.png')
print(len(vehicles))
print(len(non_vehicles))

8792
8968


In [ ]:
# Read a color image
img = cv2.imread("cutout1.jpg")
# Select a small fraction of pixels to plot by subsampling it
scale = max(img.shape[0], img.shape[1], 64) / 64  # at most 64 rows and columns
img_small = cv2.resize(img, (np.int(img.shape[1] / scale), np.int(img.shape[0] / scale)), interpolation=cv2.INTER_NEAREST)

# Convert subsampled image to desired color space(s)
img_small_RGB = cv2.cvtColor(img_small, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR, matplotlib likes RGB
img_small_HSV = cv2.cvtColor(img_small, cv2.COLOR_BGR2HSV)
img_small_LUV = cv2.cvtColor(img_small, cv2.COLOR_BGR2LUV)
img_small_HLS = cv2.cvtColor(img_small, cv2.COLOR_BGR2HLS)
img_small_YUV = cv2.cvtColor(img_small, cv2.COLOR_BGR2YUV)
img_small_YCrCb = cv2.cvtColor(img_small, cv2.COLOR_BGR2YCrCb)
img_small_rgb = img_small_RGB / 255.  # scaled to [0, 1], only for plotting

# Plot and show
plot3d(img_small_RGB, img_small_rgb)
plt.title('RGB_colorspace')
plt.show()

plot3d(img_small_HSV, img_small_rgb, axis_labels=list("HSV"))
plt.title('HSV_colorspace')
plt.show()

plot3d(img_small_LUV, img_small_rgb, axis_labels=list("LUV"))
plt.title('LUV_colorspace')
plt.show()

plot3d(img_small_HLS, img_small_rgb, axis_labels=list("HLS"))
plt.title('HLS_colorspace')
plt.show()

plot3d(img_small_YUV, img_small_rgb, axis_labels=list("YUV"))
plt.title('YUV_colorspace')
plt.show()

plot3d(img_small_YCrCb, img_small_rgb, axis_labels=list("YCrCb"))
plt.title('YCrCb_colorspace')
plt.show()

In [ ]:
# plot positives and negatives
fig, axs =plt.subplots(8,8, figsize=(16,16))
fig.subplots_adjust(hspace=.2,wspace=.001)
axs = axs.ravel()

for i in np.arange(32):
    img = cv2.imread(vehicles[np.random.randint(0,len(vehicles))])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('car', fontsize=32)
    axs[i].imshow(img)
for i in np.arange(32,64):
    img = cv2.imread(non_vehicles[np.random.randint(0,len(non_vehicles))])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axs[i].axis('off')
    axs[i].set_title('notcar', fontsize=32)
    axs[i].imshow(img)


hog() function
takes in a single color channel or grayscaled image as input, parameters are orientations, pixel_per_cell and cells_per_block
The number of orientations is specified as an integer, and represents the number of orientaion bins that the gradient information will be split into the histogram. Typical value ranges 6 to 12.
The pixels_per_cell parameters specifies the cell size over which gradient histogram is computed. This parameter is passed as a 2 tuple so you could have differnet cell sizes in x and y, but cells are commonly chosen to be squre
The cells_per_block parameter is also passed as a 2-tuple, and specified the local area over which the histogram counts in a given cell will be normalized. Block noramlization is not necessarily required, but generally leads to a more robust eature set.

In [ ]:
pos_img = cv2.imread(vehicles[np.random.randint(0, len(vehicles))])
pos_gray = cv2.cvtColor(pos_img, cv2.COLOR_BGR2GRAY)
# Define HOG parameters
orient = 9
pix_per_cell = 8
cell_per_block = 2
# Call our function with vis=True to see an image output
features, hog_image1 = get_hog_features(pos_gray, orient, 
                        pix_per_cell, cell_per_block, 
                        vis=True, feature_vec=False)


# Plot the examples

fig = plt.figure()
plt.subplot(121)
plt.imshow(pos_img, cmap='gray')
plt.title('Example Car Image')
plt.subplot(122)
plt.imshow(hog_image1, cmap='gray')
plt.title('HOG Visualization')

neg_img = cv2.imread(non_vehicles[np.random.randint(0,len(non_vehicles))])
neg_gray = cv2.cvtColor(neg_img, cv2.COLOR_BGR2GRAY)
features, hog_image2 = get_hog_features(neg_gray, orient, 
                        pix_per_cell, cell_per_block, 
                        vis=True, feature_vec=False)


# Plot the examples

fig = plt.figure()
plt.subplot(121)
plt.imshow(neg_img, cmap='gray')
plt.title('Example of non Car Image')
plt.subplot(122)
plt.imshow(hog_image2, cmap='gray')
plt.title('HOG Visualization')

In [7]:
cars = []
notcars = []
for image in vehicles:
    cars.append(image)
for image in non_vehicles:
    notcars.append(image)
print(len(cars),len(notcars))

8792 8968


In [8]:
img = cv2.imread(cars[1000])
plt.imshow(img)

In [10]:
"""
### TODO: Tweak these parameters and see how the results change.
color_space = 'RGB' # RGB # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8 #9 #8 # HOG orientations
pix_per_cell = 8 # 2 # HOG pixels per cell
cell_per_block = 2 # HOG pixels per cell
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16,16) # Spatial features on or off
hist_bins = 32 # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
"""
test_img = mpimg.imread('test_images/test6.jpg')
img_shape = test_img.shape
color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 8  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [int(img_shape[0] * 0.5), img_shape[0]] # Min and max in y to search in slide_window()
print(y_start_stop)
car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

print(len(car_features))
print(len(notcar_features))

[360, 720]
8792
8968


In [11]:
X = np.vstack((car_features, notcar_features)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)
# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
# Split up data into randomized training and test sets
rand_state = np.random.randint(0,100)
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = SVC(C=1.0, probability=True)
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

Using: 8 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 2432
698.36 Seconds to train SVC...
Test Accuracy of SVC =  0.9924


In [12]:
# Testing
n=15
print(svc.predict(X_test[100:100+n]))
print(y_test[100:100+n])

[ 0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.]
[ 1.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  0.]


In [13]:
def draw_bounding_boxes(image):
    draw_image = np.copy(image)
    hot_windows = []
    
    for window_size in [(96, 96), (120, 120)]:
        windows = slide_window(image, x_start_stop=[int(image.shape[1]/2), image.shape[1]], y_start_stop=y_start_stop, 
                            xy_window=window_size, xy_overlap=(0.9, 0.9))

        hot_windows.extend(search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat))                      

    window_img = draw_boxes(draw_image, hot_windows, color=(0, 0, 255), thick=6)                    
#     plt.imshow(window_img)
#     plt.show()
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float)

    add_heat(heat, hot_windows)

    final_map = np.clip(heat , 0, 255)
    
#     plt.imshow(final_map, cmap='hot')
#     plt.show()
    
    heatmap = apply_threshold(heat, 2)
    labels = label(heatmap)
    
    # Draw bounding boxes on a copy of the image
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    # Display the image
#     plt.imshow(draw_img)
#     plt.show()
    return draw_img
    
image_name = glob.glob("test_images/*")[2]
image = plt.imread(image_name)
plt.figure()
plt.imshow(image)
plt.show()

# image = mpimg.imread('bbox-example-image.jpg')
draw_img = draw_bounding_boxes(image)
plt.figure()
plt.imshow(draw_img)
plt.show()


In [ ]:
img = mpimg.imread('test_images/test6.jpg')
img_shape = img.shape
img_copy = np.copy(img)
y_start_stop = [int(img_shape[0] * 0.5), img_shape[0]] 
print(y_start_stop)
boxes = []
for window_size in [(96, 96), (120, 120)]:
    windows = slide_window(image, x_start_stop=[int(image.shape[1]/2), image.shape[1]], y_start_stop=y_start_stop, 
                            xy_window=window_size, xy_overlap=(0.9, 0.9))

    boxes.extend(search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat))                      

window_img = draw_boxes(img_copy, boxes, color=(0, 0, 255), thick=6)                    
plt.imshow(window_img)
plt.show()
heat = np.zeros_like(image[:,:,0]).astype(np.float)
add_heat(heat, boxes)
final_map = np.clip(heat , 0, 255)
plt.imshow(final_map, cmap='hot')
plt.show()
heatmap = apply_threshold(heat, 2)
labels = label(heatmap)
# Draw bounding boxes on a copy of the image
draw_img = draw_labeled_bboxes(np.copy(image), labels)
# Display the image
plt.imshow(draw_img)
plt.show()

In [14]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [15]:
def process(image):
    return draw_bounding_boxes(image)


In [16]:
for image_name in glob.glob("test_images/*"):
    test_img = plt.imread(image_name)
    test_img = process(test_img)
    plt.imshow(test_img)
    plt.show()

In [ ]:
output = 'p5.mp4'
clip1 = VideoFileClip("./project_video.mp4")
output_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
%time output_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video p5.mp4
[MoviePy] Writing video p5.mp4


  8%|▊         | 104/1261 [1:15:04<14:12:14, 44.20s/it]